# Model Iteration 1
Here, I want to implement a very simple model. To do this, I'm not going to create very many features, just recode some of the variables that are categorical like district. 

## Importing Everything!

In [61]:
import pandas as pd
import numpy as np
import itertools
import re

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
from datetime import datetime
from ipywidgets import widgets  
from IPython.display import display


from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.metrics import log_loss

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

print "imports done"

from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint


imports done
building tree 108 of 200








building tree 108 of 200building tree 108 of 200
building tree 108 of 200building tree 310 of 400building tree 310 of 400building tree 310 of 400building tree 310 of 400building tree 310 of 400building tree 310 of 400building tree 310 of 400building tree 310 of 400

building tree 109 of 200








building tree 109 of 200building tree 109 of 200
building tree 109 of 200building tree 311 of 400building tree 311 of 400building tree 311 of 400building tree 311 of 400building tree 311 of 400building tree 311 of 400building tree 311 of 400building tree 311 of 400

building tree 110 of 200








building tree 110 of 200building tree 110 of 200
building tree 110 of 200building tree 312 of 400building tree 312 of 400building tree 312 of 400building tree 312 of 400building tree 312 of 400building tree 312 of 400building tree 312 of 400building tree 312 of 400

building tree 111 of 200








building tree 111 of 200building tree 111 of 200
build

## Loading in the data
Now, I need to load in the data

In [3]:
readData = pd.read_csv('train.csv')

print "dataRead"

Categories = ['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
              'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
              'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
              'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
              'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
              'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
              'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
              'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
              'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
              'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
              'WARRANTS', 'WEAPON LAWS']

dataRead


## Helper functions for recoding data
Here are the helper functions for recoding data. We'll add more as we create some new features

In [4]:
def recodeData(df, isTrain = False):
    '''This function takes in the dataframe that we get from loading in the 
    SF crime data and returns a re-coded dataframe that has all the 
    additional features we want to add and the categorical features recoded 
    and cleaned.
    '''

    #since the modifications are done in-place we don't return the dataframe. 
    #we do, however, return the list of all the columns we added.
    df, newLatLon = removeOutlierLatLon(df)
    df, newDate = recodeDates(df)
    df, newDistrict = recodePoliceDistricts(df)
    print "recoding addresses"
    df, newAddress, streetColumns = recodeAddresses(df)

    
    addedColumns = [] 
    addedColumns += newDate
    addedColumns += newDistrict 
    addedColumns += newLatLon
    addedColumns += newAddress
    
    
   

    if (isTrain):
        newCategory = recodeCategories(df)
        addedColumns += newCategory
        try: #prevents error if the coumns have already been removed or we are processing test data
            columnsToDrop = ['Descript', 'Resolution']
            df.drop(columnsToDrop, axis=1, inplace=True)
        except:
            print "already recoded or using test data"
         

    return df, addedColumns, streetColumns

In [5]:
def recodeDates(df):
    '''This function takes in a dataframe and recodes the date field into 
    useable values. Here, we also recode the day of week.'''
    #Recode the dates column to year, month, day and hour columns
    df['DateTime'] = df['Dates'].apply(
        lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

    df['Year'] = df['DateTime'].apply(lambda x: x.year)
    df['Month'] = df['DateTime'].apply(lambda x: x.month)
    df['Day'] = df['DateTime'].apply(lambda x: x.day)
    df['Hour'] = df['DateTime'].apply(lambda x: x.hour)
    df['Minute'] = df['DateTime'].apply(lambda x: x.minute)
    df['DayOfWeekRecode'] = df['DateTime'].apply(lambda x: x.weekday())
    #df['MinuteOfWeek'] = df['DateTime'].apply(lambda x: x.weekday()*24*60 + x.hour*60 + x.minute)

    return df, ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DayOfWeekRecode']

In [6]:
def recodePoliceDistricts(df):
    '''This function recodes the police district to a one-hot encoding 
    scheme.'''
    districts = df['PdDistrict'].unique().tolist()
    newColumns = []
    for district in districts:
        newColumns.append('District' + district)
        df['District' + district] = df['PdDistrict'].apply(
            lambda x: int(x == district))

    return df, newColumns

In [7]:
def recodeAddresses(df):
    '''This function will attempt to create some features related to the address field in the database. To do this, 
    first, we need to split up the address field into two different address fields'''
    
    #If there are two addresss, split fields. Also extract the block number
    df['Address1'] = df['Address'].apply(lambda x: re.sub(r'^\d+ Block of ','',x.split(" / ")[0]))
    df['Address2'] = df['Address'].apply(lambda x: (x.split(" / ")[1]) if (len(x.split(" / ")) > 1) else '')
    
    streets = set(df['Address1'].unique().tolist() + df['Address2'].unique().tolist())
    
    streetColumns = []
    i = 0
    
    print "starting address dummy creation"
    
    #address1Dummy = pd.get_dummies(df['Address1']).rename(columns=lambda x: str(x))
    address1Dummy = pd.get_dummies(df['Address1'])
    address1Dummy = address1Dummy.replace(0, np.nan)
    
    print "completed address 1 dummy creation"
    address2Dummy = pd.get_dummies(df['Address2'])
    address2Dummy = address2Dummy.replace(0, np.nan)
    
    print "completed address 2 dummy creation"
    
#     print address1Dummy
#     print address2Dummy.info()
    
    mergedAddressDummy = address1Dummy.combine_first(address2Dummy)
    
    print "completed address dummy DataFrames merge"
    mergedAddressDummy = mergedAddressDummy.fillna(0)
    print "completed fillna on mergedAddressDummy"
    
    streetColumns = list(mergedAddressDummy.columns.values)
    
    df = pd.concat([df, mergedAddressDummy], axis=1)
    print "completed merge of original df and new dummy variable df"
    
    print "Dummy creation finished"
    
#     for street in streets:
#         df['OnStreet' + street] = df.apply(lambda x: (x['Address1'] == street or x['Address2'] == street), axis=1)
#         streetColumns.append('OnStreet' + street)
#         i += 1
#         print i
        

    
    df['BlockNumber'] = df['Address'].apply(lambda x: int(re.findall(r'^\d+',x)[0]) if (len(re.findall(r'^\d+',x)) > 0) else None )
    df['BlockNumber'] = df['BlockNumber'].fillna(-1)
    
    #Also add the "did the crime occur on a street corner field?"
    df['StreetCornerFlag'] = df['Address'].apply(lambda x: len(x.split(" / ")) > 1)
    
    return df, ['StreetCornerFlag', 'BlockNumber'], streetColumns

In [8]:
def removeOutlierLatLon(df):
    '''This function will attempt remove outlier Latitudes and Longitudes'''
    df.loc[df.X > -121, 'X'] = df.loc[(df.X > -121)].apply(lambda row: df.X[df["PdDistrict"] == row['PdDistrict']].median(), axis=1)
    df.loc[df.Y > 38, 'Y'] = df.loc[(df.Y > 38)].apply(lambda row: df.Y[df["PdDistrict"] == row['PdDistrict']].median(), axis=1)

    return df, ['X', 'Y']

In [9]:
def recodeCategories(df):
    '''This function will attempt remove outlier Latitudes and Longitudes'''
    #if 'Category' in df.columns:
    df['CategoryRecode'] = df.Category.apply(lambda x: Categories.index(x))
        
    return df, ['CategoryRecode']

## Recoding Columns
Here, we want to do some recoding of the columns. To do this, we're going to use our handy-dandy helper functions.  

In [10]:
crimeData, addedColumns, streetColumns = recodeData(
    readData, isTrain = True)
crimeData.describe()


recoding addresses
starting address dummy creation
completed address 1 dummy creation
completed address 2 dummy creation
completed address dummy DataFrames merge
completed fillna on mergedAddressDummy
completed merge of original df and new dummy variable df
Dummy creation finished


/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

,X,Y,Year,Month,Day,Hour,Minute,DayOfWeekRecode,DistrictNORTHERN,DistrictPARK,...,YOUNG CT,YUKON ST,ZAMPA LN,ZENO PL,ZIRCON PL,ZOE ST,ZOO RD,BlockNumber,StreetCornerFlag,CategoryRecode
count,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,...,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049,878049.000000
mean,-122.422763,37.767035,2008.712046,6.436509,15.570623,13.412655,20.155026,2.992691,0.119920,0.056162,...,0.000026,0.000026,0.000018,0.000005,0.000015,0.000091,0.000114,704.776747,0.2970381,19.338687
std,0.025285,0.024165,3.631194,3.428972,8.783005,6.549573,18.594915,1.972023,0.324869,0.230234,...,0.005118,0.005118,0.004269,0.002134,0.003848,0.009545,0.010671,1001.193208,0.4569537,10.688637
min,-122.513642,37.707879,2003.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,False,0.000000
25%,-122.432952,37.752427,2006.000000,3.000000,8.000000,9.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,16.000000
50%,-122.416420,37.775421,2009.000000,6.000000,16.000000,14.000000,19.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,300.000000,0,20.000000
75%,-122.406959,37.784368,2012.000000,9.000000,23.000000,19.000000,33.000000,5.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1000.000000,1,25.000000
max,-122.364937,37.819975,2015.000000,12.000000,31.000000,23.000000,59.000000,6.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8300.000000,True,38.000000


In [10]:
streetColumns

['',
 ' HARRISON ST',
 '10TH AV',
 '10TH ST',
 '11TH AV',
 '11TH ST',
 '12TH AV',
 '12TH ST',
 '13TH ST',
 '14TH AV',
 '14TH ST',
 '15TH AV',
 '15TH ST',
 '16TH AV',
 '16TH ST',
 '17TH AV',
 '17TH ST',
 '18TH AV',
 '18TH ST',
 '19TH AV',
 '19TH ST',
 '1ST ST',
 '20TH AV',
 '20TH ST',
 '21ST AV',
 '21ST ST',
 '22ND AV',
 '22ND ST',
 '23RD AV',
 '23RD ST',
 '24TH AV',
 '24TH ST',
 '25TH AV',
 '25TH ST',
 '26TH AV',
 '26TH ST',
 '27TH AV',
 '27TH ST',
 '28TH AV',
 '28TH ST',
 '29TH AV',
 '29TH ST',
 '2ND AV',
 '2ND ST',
 '30TH AV',
 '30TH ST',
 '31ST AV',
 '32ND AV',
 '33RD AV',
 '34TH AV',
 '35TH AV',
 '36TH AV',
 '37TH AV',
 '38TH AV',
 '39TH AV',
 '3RD AV',
 '3RD ST',
 '40TH AV',
 '41ST AV',
 '42ND AV',
 '43RD AV',
 '44TH AV',
 '45TH AV',
 '46TH AV',
 '47TH AV',
 '48TH AV',
 '4TH AV',
 '4TH ST',
 '5TH AV',
 '5TH ST',
 '5THSTNORTH ST',
 '6TH AV',
 '6TH ST',
 '7TH AV',
 '7TH ST',
 '7THSTNORTH ST',
 '8TH AV',
 '8TH ST',
 '9TH AV',
 '9TH ST',
 'ABBEY ST',
 'ACADIA ST',
 'ACEVEDO AV',
 'ACT

In [11]:
crimeData.head(5)

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,DateTime,Year,Month,...,YOUNG CT,YUKON ST,ZAMPA LN,ZENO PL,ZIRCON PL,ZOE ST,ZOO RD,BlockNumber,StreetCornerFlag,CategoryRecode
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,2015,5,...,0,0,0,0,0,0,0,-1,True,37
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13 23:53:00,2015,5,...,0,0,0,0,0,0,0,-1,True,21
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015-05-13 23:33:00,2015,5,...,0,0,0,0,0,0,0,-1,True,21
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13 23:30:00,2015,5,...,0,0,0,0,0,0,0,1500,False,16
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13 23:30:00,2015,5,...,0,0,0,0,0,0,0,100,False,16


In [13]:
commonStreets = ['FOLSOM ST',
 '16TH ST',
 'JONES ST',
 'TAYLOR ST',
 'ARMSTRONG AV',
 'EDDY ST',
 'LARKIN ST',
 'CASTRO ST',
 '10TH AV',
 '5TH ST',
 'HAIGHT ST',
 'OFARRELL ST',
 '11TH AV',
 'PAGE ST',
 'FITCH ST',
 'CAPP ST',
 '13TH ST',
 '24TH AV',
 '17TH ST',
 '18TH ST',
 '19TH ST',
 'GENEVA AV',
 'GEARY BL',
 'BRYANT ST',
 'HYDE ST',
 '4TH ST',
 'FULTON ST',
 'LEAVENWORTH ST',
 'COLE ST',
 'ALEMANY BL',
 'PHELPS ST',
 'MISSION ST',
 '6TH ST',
 '12TH AV',
 'SHOTWELL ST',
 'TREAT AV',
 '7TH ST',
 'JEFFERSON ST',
 'QUESADA AV',
 'TURK ST',
 '2ND ST',
 'MARKET ST',
 'GGBRIDGE HY',
 '24TH ST',
 'CAPITOL AV',
 'KEARNY ST',
 'HARRISON ST',
 'LYON ST',
 'BUSH ST',
 'POLK ST',
 '3RD ST',
 'ELLIS ST',
 'SOUTH VAN NESS AV',
 'POTRERO AV',
 '20TH ST',
 'POWELL ST']

## Model Iteration 1
Now that I've done some recoding, I'm going to create my model. To do this, I'm going to do a random forest classifier. 

In [17]:
columnsToUse = addedColumns

columnsToUse = ['X','Y', 'Year', 'Month', 'Hour', 'Minute',
       'DayOfWeekRecode', 'DistrictNORTHERN', 'DistrictPARK',
       'DistrictINGLESIDE', 'DistrictBAYVIEW', 'DistrictRICHMOND',
       'DistrictCENTRAL', 'DistrictTARAVAL', 'DistrictTENDERLOIN',
       'DistrictMISSION', 'DistrictSOUTHERN', 'StreetCornerFlag', 'BlockNumber']



X = crimeData[columnsToUse]
y = crimeData['CategoryRecode']

clf = RandomForestClassifier(n_estimators=400, max_depth = 20, random_state=1, n_jobs = 1, verbose=2)

In [13]:


k_folds = StratifiedShuffleSplit(y, 3, test_size=0.5, random_state=0)

scores = []
print "starting kfold"

for k, (train, test) in enumerate(k_folds):
    clf.fit(X.iloc[train], y.iloc[train])
    probs = clf.predict_proba(X.iloc[test])
    score = log_loss(y.iloc[test].values, probs)
    print score
    scores.append(score)
    
print(scores)
print("Average: " + str(np.average(scores)))

starting kfold
building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building t

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed: 20.9min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 20.9min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  2.2min



2.28301178819
building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building t

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed: 18.8min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.8min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  2.3min



2.28332215178
building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building t

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed: 19.1min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 19.1min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  2.1min



2.28263089497
[2.2830117881936989, 2.2833221517807698, 2.2826308949675145]
Average: 2.28298827831


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min finished


In [18]:
def make_submission(clf, Xtrain, ytrain, predictors, path='my_submission.csv'):
    '''This function will take in a trained model, a list of predictors, and an optional 
    filepath and create a submissision file for us.'''
   
    test_data = pd.read_csv('test.csv')
    print "test data loaded"
    
    test_data, newColumns, streetColumns = recodeData(test_data)
    print "test data recoded"
    
    testDataColumns = list(test_data.columns.values)
    
    existingPredictors = list(set(predictors) & set(testDataColumns))
    
    clf.fit(Xtrain[existingPredictors], ytrain)
    print "model fitted with all data"
    
    #clf.fit(trainX[predictors], trainY)
    predictions = clf.predict_proba(test_data[existingPredictors])

    submission = pd.DataFrame({
        'Id': test_data.Id
    })
    
    for i in range(predictions.shape[1]):
        submission[Categories[i]] = predictions[:,i]
    submission.to_csv(path, index=False)

    print(" -- Wrote submission to file {}.".format(path))
    return existingPredictors


In [26]:
predictorsUsed = make_submission(clf, X, y, columnsToUse)

test data loaded
recoding addresses
starting address dummy creation
completed address 1 dummy creation
completed address 2 dummy creation
completed address dummy DataFrames merge
completed fillna on mergedAddressDummy
completed merge of original df and new dummy variable df
Dummy creation finished
test data recoded
building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed: 42.2min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 42.2min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:  4.1min



 -- Wrote submission to file my_submission.csv.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.1min finished


In [28]:
predictorsUsedSaved = ['FOLSOM ST',
 '16TH ST',
 '13TH ST',
 'TAYLOR ST',
 'ARMSTRONG AV',
 'EDDY ST',
 'TREAT AV',
 'LARKIN ST',
 'CASTRO ST',
 'StreetCornerFlag',
 '10TH AV',
 '5TH ST',
 'HAIGHT ST',
 'DistrictNORTHERN',
 'OFARRELL ST',
 'QUESADA AV',
 '11TH AV',
 'DistrictPARK',
 'PAGE ST',
 'FITCH ST',
 'DistrictTARAVAL',
 'JONES ST',
 'GEARY BL',
 '17TH ST',
 '18TH ST',
 '19TH ST',
 'BRYANT ST',
 'HYDE ST',
 '4TH ST',
 'GENEVA AV',
 'Minute',
 'COLE ST',
 '7TH ST',
 'DistrictBAYVIEW',
 'ALEMANY BL',
 'PHELPS ST',
 'MISSION ST',
 'Hour',
 '6TH ST',
 'DistrictSOUTHERN',
 'CAPP ST',
 'SHOTWELL ST',
 '12TH AV',
 'DistrictTENDERLOIN',
 'DayOfWeekRecode',
 'JEFFERSON ST',
 'Month',
 'Y',
 'X',
 'TURK ST',
 '2ND ST',
 'MARKET ST',
 'GGBRIDGE HY',
 'DistrictCENTRAL',
 'DistrictINGLESIDE',
 '24TH AV',
 '24TH ST',
 'CAPITOL AV',
 'KEARNY ST',
 'HARRISON ST',
 'FULTON ST',
 'BUSH ST',
 'POLK ST',
 'DistrictRICHMOND',
 'BlockNumber',
 '3RD ST',
 'ELLIS ST',
 'SOUTH VAN NESS AV',
 'LEAVENWORTH ST',
 'POTRERO AV',
 'Year',
 '20TH ST',
 'DistrictMISSION',
 'POWELL ST',
 'LYON ST']


Best Score: 	
2.44156

In [16]:
countsByStreet = crimeData.groupby(['Category','Address1']).count();
countsByStreet[countsByStreet.Dates > 100];

Dates  DayOfWeek  PdDistrict  Address     X  \
Category    Address1                                                         
ASSAULT     10TH ST              106        106         106      106   106   
            11TH ST              151        151         151      151   151   
            14TH ST              134        134         134      134   134   
            15TH ST              162        162         162      162   162   
            16TH ST              991        991         991      991   991   
            17TH ST              293        293         293      293   293   
            18TH ST              571        571         571      571   571   
            19TH AV              189        189         189      189   189   
            19TH ST              178        178         178      178   178   
            20TH AV              157        157         157      157   157   
            20TH ST              174        174         174      174   174   
            21ST ST              173        173         173      173   173   
            22ND ST              215        215         215      215   215   
            23RD ST              281        281         281      281   281   
            24TH AV              171        171         171      171   171   
            24TH ST              618        618         618      618   618   
            25TH ST              302        302         302      302   302   
            26TH ST              249        249         249      249   249   
            2ND ST               129        129         129      129   129   
            32ND AV              133        133         133      133   133   
            3RD ST              1228       1228        1228     1228  1228   
            46TH AV              102        102         102      102   102   
            4TH ST               371        371         371      371   371   
            5TH ST               351        351         351      351   351   
            6TH ST              1233       1233        1233     1233  1233   
            7TH ST               307        307         307      307   307   
            8TH ST               217        217         217      217   217   
            9TH ST               222        222         222      222   222   
            ALABAMA ST           129        129         129      129   129   
            ALEMANY BL           194        194         194      194   194   
...                              ...        ...         ...      ...   ...   
WARRANTS    PALOU AV             130        130         130      130   130   
            PINE ST              137        137         137      137   137   
            POLK ST              546        546         546      546   546   
            POST ST              352        352         352      352   352   
            POTRERO AV           458        458         458      458   458   
            POWELL ST            374        374         374      374   374   
            SAN BRUNO AV         155        155         155      155   155   
            SHOTWELL ST          168        168         168      168   168   
            SOUTH VAN NESS AV    227        227         227      227   227   
            STANYAN ST           311        311         311      311   311   
            STEVENSON ST         214        214         214      214   214   
            STOCKTON ST          171        171         171      171   171   
            SUNNYDALE AV         147        147         147      147   147   
            SUTTER ST            233        233         233      233   233   
            TAYLOR ST            379        379         379      379   379   
            TURK ST             1433       1433        1433     1433  1433   
            UNITEDNATIONS PZ     225        225         225      225   225   
            VALENCIA ST          297        297         297      297   297   
            VANNESS AV           267        267      

In [54]:
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

clfOpt = RandomForestClassifier(verbose=3)
#(n_estimators=400, max_depth = 20, random_state=1, )

param_dist = {"max_depth": [5, 10, 20 , None],
              "n_estimators": [50, 200, 400, 800],
              "min_samples_split": [2, 10, 100],
              "max_features": ['auto', None]}

# run randomized search
n_iter_search = 48
random_search = RandomizedSearchCV(clfOpt, param_distributions=param_dist,
                                   n_iter=n_iter_search, scoring="log_loss", verbose=3, n_jobs = 12)

finalX = X[predictorsUsedSaved].values.astype(np.float32)
finaly = y.values.astype(int)
print "pre Random search finished"

pre Random search finished
building tree 96 of 400building tree 96 of 400building tree 96 of 400building tree 96 of 400building tree 96 of 400building tree 96 of 400











building tree 97 of 200building tree 97 of 200building tree 97 of 200building tree 97 of 200building tree 97 of 400building tree 97 of 400building tree 97 of 400building tree 97 of 400building tree 97 of 400building tree 97 of 400building tree 97 of 400building tree 97 of 400











building tree 98 of 200building tree 98 of 200building tree 98 of 200building tree 98 of 200building tree 98 of 400building tree 98 of 400building tree 98 of 400building tree 98 of 400building tree 98 of 400building tree 98 of 400building tree 98 of 400building tree 98 of 400











building tree 99 of 200building tree 99 of 200building tree 99 of 200building tree 99 of 200building tree 99 of 400building tree 99 of 400building tree 99 of 400building tree 99 of 400building tree 99 of 400building tree 99 of 400building tree 99 

In [ ]:
print "starting Randomized Search"
start = time()
random_search.fit(finalX, finaly)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)

starting Randomized Search
Fitting 2 folds for each of 48 candidates, totalling 96 fits


In [50]:
finalXfloat = finalX.astype(np.float32)

In [42]:
finalXfloat.dtype

dtype('float64')

In [51]:
finalXfloat.nbytes

263414700